In [44]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import sklearn
import keras.backend as K
from keras.models import Sequential
from keras.layers import Dense
from sklearn.ensemble import RandomForestRegressor
from sklearn import linear_model
from sklearn import svm
import scipy.io
import pickle

In [74]:
def parse_mat_matrix(filename, category, normalization=False, filter_light_isos=True):
    f = open(filename, 'r')
    lines = f.readlines()
    read = False
    for indx, line in enumerate(lines):
        if ';' in line:
            read = False
        if read:
            vector = line.split(' ')
            # parse out %
            comment_indx = vector.index('%')
            vector = vector[:comment_indx]
            matrix = np.vstack((matrix, vector))
        if category in line:
            read = True
            vector = lines[indx+1].split(' ')
            comment_indx = vector.index('%')
            length = len(vector[:comment_indx])
            matrix = np.empty(length)
    # get rid of first line because it's the empty one
    # get rid of the last two because they are lost and total
    matrix = np.array(matrix[1:], dtype='float64')[:-2]
    
    if filter_light_isos:
        matrix[:15, :] = np.zeros((15, np.shape(matrix)[1]))
    
    if normalization:
        new_matrix = np.empty(np.shape(matrix))
        for i in range(np.shape(matrix)[1]):
            new_matrix[:,i] = normalize(matrix[:,i])
        return new_matrix
    # no lost and total
    return matrix


def parse_mat_name(filename):
    f = open(filename, 'r')
    lines = f.readlines()
    read = False
    names = []
    for line in lines:
        if ';' in line:
            read = False
        if read:
            iso = line.replace(' ', '').replace("'", '').strip()
            names.append(iso)
        if 'NAMES' in line:
            read = True
    # get rid of lost and total
    return names[:-2]

def parse_mat_burnup(filename):
    f = open(filename, 'r')
    lines = f.readlines()
    for line in lines:
        if 'TOT_BURNUP' in line:
            bu = line.split()
            a = bu.index('[') + 1
            z = bu.index('];')
            bu = bu[a:z]
    return np.array(bu, dtype='float64') * 1e3

def name_and_matrix_to_dict(name_list, matrix):
    return_dict = {}
    for indx, val in enumerate(name_list):
        return_dict[val] = matrix[indx]
    return return_dict

def normalize(array):
    new_array = np.empty(len(array))
    for indx,val in enumerate(array):
        new_array[indx] = val / sum(array)
    return new_array


## Automatically collect SERPENT dep file into pd.DataFrame

In [75]:
# do it for all dep files
import os
subdirectory_list = [x[0] for x in os.walk('./serpent_test_burnup')][1:]
dir_dict = {}
for subdir in subdirectory_list:
    enr = subdir.split('_')[-1]
    enr = enr.replace('%', '')
    dir_dict[enr] =  subdir
dir_dict

{'3.3': './serpent_test_burnup/case_1_enrichment_3.3%',
 '4.9': './serpent_test_burnup/case_10_enrichment_4.9%',
 '4.1': './serpent_test_burnup/case_6_enrichment_4.1%',
 '3.2': './serpent_test_burnup/case_7_enrichment_3.2%',
 '4.7': './serpent_test_burnup/case_5_enrichment_4.7%',
 '4.6': './serpent_test_burnup/case_2_enrichment_4.6%',
 '3.5': './serpent_test_burnup/case_9_enrichment_3.5%',
 '3.9': './serpent_test_burnup/case_3_enrichment_3.9%',
 '3.7': './serpent_test_burnup/case_4_enrichment_3.7%',
 '3.6': './serpent_test_burnup/case_8_enrichment_3.6%'}

In [76]:
coll_dict = {}
first = True
for enr, path in dir_dict.items():
    filename = path + '/pwr_dep.m'
    name_matrix_dict = name_and_matrix_to_dict(name_list = parse_mat_name(filename),
                        matrix = parse_mat_matrix(filename, 'MAT_UO2_MDENS',
                                                  normalization=True,
                                                  filter_light_isos=True))
    name_matrix_dict['bu'] = parse_mat_burnup(filename)
    name_matrix_dict['enr'] = [enr] * len(name_matrix_dict['bu'])
    coll_dict[enr] = name_matrix_dict

In [77]:
enr_num = len(dir_dict.keys())
bu_num = len(parse_mat_burnup(filename))
print('TOTAL NUMBER OF DEPLETION CALCULATIONS SHOULD BE: \n'
      '[enrichment variation * burnup steps]\n'
      '%i * %i = %i'
%(enr_num, bu_num, enr_num * bu_num))

TOTAL NUMBER OF DEPLETION CALCULATIONS SHOULD BE: 
[enrichment variation * burnup steps]
10 * 21 = 210


In [78]:
df_dict = {}
first_key = list(coll_dict.keys())[0]
for category in coll_dict[first_key].keys():
    long_vec = np.empty(0)
    for key, val in coll_dict.items():
        long_vec = np.append(long_vec, val[category])
    df_dict[category] = long_vec

In [79]:
df = pd.DataFrame.from_dict(df_dict)

## Change column name to fit udb data

In [80]:
def insert_hyphen(string):
    for indx, i in enumerate(string):
        if i.isdigit():
            site = indx
            break
    with_hyphen = string[:site] + '-' + string[site:]
    return with_hyphen.lower()

def does_it_have_number(string):
    for i in string:
        if i.isdigit():
            return True
    return False

change_dict = {}
for i in list(df):
    if does_it_have_number(i):
        change_dict[i] = insert_hyphen(i)
    else:
        change_dict[i] = i

change_dict['enr'] = 'init_enr'

df =df.rename(index=str, columns = change_dict)

## Export UDB and SCALE depletion calculations as file

In [81]:
all_dat = pd.read_csv('./curated.csv', index_col=0)

# sift out pwrs
all_dat = all_dat.loc[all_dat['reactor_type'] == 'PWR']  
all_dat = sklearn.utils.shuffle(all_dat)
# only get assemblies with enrichment bigger than 1.5 and bunrup higher than 10,000
all_dat = all_dat.loc[(all_dat['init_enr'] > 1.5) & (all_dat['bu'] > 10000)]

data_dict = {'UDB': all_dat, 'SERPENT': df}

with open('all_data.pkl', 'wb') as f:
    pickle.dump(data_dict, f)
